# Is Google biased in Representing Gun-related Issues in the US?
(Will be filled from the [Google Doc](https://docs.google.com/document/d/1Vn5gixfqObaT2fnBjv8SkHH2RwmZA7VP6_rtFlOs1go/edit#))

# Methodology


In [3]:
from googlesearch import search_news
for url in search_news('gun', stop=20):
    print(url)


https://www.la-croix.com/Monde/Japon-femme-pilote-chasse-premiere-fois-2018-08-23-1300963468
https://www.ladepeche.fr/article/2018/04/25/2786889-78-ans-moroder-roi-disco-tire-reverence.html
https://www.lexpress.fr/actualites/1/monde/pour-les-experts-de-l-oiac-mission-quasi-impossible-a-douma_2000759.html
https://www.ladepeche.fr/article/2016/07/05/2379329-de-bruxelles-a-hollywood-l-aventure-des-realisateurs-de-black.html
https://wpolityce.pl/polityka/139280-badaczka-krystyna-piorkowska-o-amerykanskich-dokumentach-ws-katynia
https://www.geenstijl.nl/4614731/quiz_wat_heeft_die_oranje_gori/
https://www.ouest-france.fr/europe/belgique/belgique-un-homme-arrete-pour-avoir-tente-de-foncer-sur-des-pietons-4878660
https://www.geenstijl.nl/4613981/oranje_komt_in_verzet/
https://forsal.pl/artykuly/944914,dyplomata-usa-o-rosyjskich-prowokacjach-nie-potrzebujemy-powtorki-z-top-gun-nad-baltykiem.html
https://www.panorama.it/musica/i-kiss-allarena-verona-l11-giugno/
